In [3]:
# inspect dataset

In [2]:
!pip install geopandas
!pip install shapely

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.validation import make_valid
import numpy as np
import matplotlib.pyplot as plt

### Inspecting Allegheny County census tract file for future validation purposes. 

Data taken from WPRDC should show all Census Tracts in year 2016. Result: there should be 402 rows in our final data set. 

In [189]:
alCoCTs = pd.read_csv('AlleghenyCountyCensusTracts2016.csv')
print('Number of Census Tracts: ', len(alCoCTs))
alCoCTs.sort_values('geoid')

Number of Census Tracts:  402


,_id,fid,state_fp,county_fp,tract_ce,affgeoid,geoid,name,lsad,a_land,a_water,shape_length,shape_area
69,15019,70,42,3,10300,1400000US42003010300,42003010300,103.0,CT,866087,235982,0.055235,0.000117
368,15318,369,42,3,20100,1400000US42003020100,42003020100,201.0,CT,1678102,483177,0.083161,0.000231
28,14978,29,42,3,20300,1400000US42003020300,42003020300,203.0,CT,1594790,326537,0.077032,0.000204
70,15020,71,42,3,30500,1400000US42003030500,42003030500,305.0,CT,668309,0,0.046956,0.000072
394,15344,395,42,3,40200,1400000US42003040200,42003040200,402.0,CT,604717,0,0.049916,0.000064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,15162,213,42,3,981000,1400000US42003981000,42003981000,9810.0,CT,1372534,0,0.063284,0.000147
80,15030,81,42,3,981100,1400000US42003981100,42003981100,9811.0,CT,875989,0,0.043523,0.000093
68,15018,69,42,3,981200,1400000US42003981200,42003981200,9812.0,CT,460275,156913,0.037880,0.000067
165,15115,166,42,3,981800,1400000US42003981800,42003981800,9818.0,CT,1822889,310071,0.066922,0.000228


In [55]:
cities2019 = pd.read_csv("500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release_20250405.csv")
cities2019=cities2019[cities2019['PlaceName'] == 'Pittsburgh']
cities2019['Year'] = ['2019']*len(cities2019)

cities2018 = pd.read_csv("500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2018_release_20250416.csv")
cities2018=cities2018[cities2018['PlaceName'] == 'Pittsburgh']
cities2018['Year'] = ['2018']*len(cities2018)

cities2017 = pd.read_csv("500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2017_release_20250416.csv")
cities2017=cities2017[cities2017['PlaceName'] == 'Pittsburgh']
cities2017['Year'] = ['2017']*len(cities2017)

cities2016 = pd.read_csv("500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2016_release_20250416.csv")
cities2016=cities2016[cities2016['PlaceName'] == 'Pittsburgh']
cities2016['Year'] = ['2016']*len(cities2016)

cond_prevn = pd.concat([cities2019, cities2018, cities2017, cities2016])
cond_prevn

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,Year,population_count
450,PA,Pittsburgh,4261000,42003010300,4261000-42003010300,6600.0,12.9,"(10.2, 16.9)",11.5,"(10.9, 12.3)",...,"( 8.8, 11.8)",42.3,"(39.5, 44.4)",1.6,"( 1.5, 1.8)",24.3,"(19.1, 30.4)","(40.43572601140, -79.9851241394)",2019,NaN
451,PA,Pittsburgh,4261000,42003020100,4261000-42003020100,3629.0,6.7,"( 5.5, 8.5)",17.7,"(17.0, 18.4)",...,"( 7.2, 8.7)",35.9,"(34.0, 37.8)",1.9,"( 1.8, 2.1)",11.0,"( 8.4, 13.8)","(40.44068353060, -80.0000351595)",2019,NaN
452,PA,Pittsburgh,4261000,42003020300,4261000-42003020300,616.0,4.8,"( 4.1, 5.8)",16.8,"(16.2, 17.5)",...,"( 5.9, 7.2)",34.6,"(33.4, 36.1)",1.4,"( 1.3, 1.5)",5.3,"( 4.0, 7.5)","(40.45425423370, -79.9814998549)",2019,NaN
453,PA,Pittsburgh,4261000,42003030500,4261000-42003030500,2256.0,14.4,"(11.9, 17.2)",35.6,"(34.1, 37.0)",...,"(16.8, 20.8)",47.5,"(45.9, 49.0)",7.1,"( 6.2, 8.1)",30.2,"(21.5, 39.5)","(40.44273346300, -79.9837719110)",2019,NaN
454,PA,Pittsburgh,4261000,42003040200,4261000-42003040200,2604.0,11.8,"( 8.8, 15.8)",16.5,"(15.7, 17.4)",...,"( 9.8, 13.4)",41.2,"(37.8, 44.0)",2.8,"( 2.5, 3.0)",27.7,"(19.6, 36.3)","(40.44011022030, -79.9663476233)",2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21696,PA,Pittsburgh,4261000,42003562800,4261000-42003562800,1597.0,13.3,"(11.4, 15.2)",31.8,"(30.8, 32.9)",...,"(11.7, 14.0)",38.2,"(36.9, 39.4)",3.4,"( 3.0, 3.8)",16.7,"(12.0, 22.5)","(40.4226894882, -80.0713440743)",2016,NaN
21700,PA,Pittsburgh,4261000,42003310300,4261000-42003310300,990.0,9.5,"( 7.7, 11.3)",31.8,"(30.4, 33.2)",...,"( 9.8, 12.5)",36.3,"(34.6, 37.9)",3.1,"( 2.7, 3.4)",12.8,"( 8.7, 17.6)","(40.3879203774, -79.9225535388)",2016,NaN
21701,PA,Pittsburgh,4261000,42003050900,4261000-42003050900,1202.0,36.3,"(30.9, 42.0)",39.1,"(38.0, 40.4)",...,"(23.7, 28.2)",53.9,"(51.9, 55.5)",8.7,"( 7.8, 9.7)",45.2,"(36.4, 54.4)","(40.4510464389, -79.9743932785)",2016,NaN
21702,PA,Pittsburgh,4261000,42003070600,4261000-42003070600,2082.0,7.5,"( 6.3, 9.2)",17.1,"(16.6, 17.7)",...,"( 5.4, 6.4)",32.5,"(31.2, 33.9)",1.4,"( 1.3, 1.5)",7.1,"( 5.2, 10.0)","(40.4532703527, -79.9254775124)",2016,NaN


In [57]:
cond_prevn = cond_prevn.rename(columns={'TractFIPS': 'Geo_FIPS'})
#Removing unrelated prevalence columns (not comorbidities with Asthma)
def drop_columns_with_keyword(df, keyword):
    # Identify columns containing the keyword (95CI)
    cols_to_drop = [col for col in df.columns if keyword.lower() in col.lower()]
    # Drop them and return the updated DataFrame
    return df.drop(columns=cols_to_drop)
cond_prevn = drop_columns_with_keyword(cond_prevn,'95CI')
cond_prevn = cond_prevn.drop([
    'ARTHRITIS_CrudePrev',
    'BINGE_CrudePrev',
    'CANCER_CrudePrev',
    'COLON_SCREEN_CrudePrev',
    'DENTAL_CrudePrev',
    'KIDNEY_CrudePrev',
    'LPA_CrudePrev',
    'MAMMOUSE_CrudePrev',
    'MHLTH_CrudePrev',
    'PAPTEST_CrudePrev',
    'PHLTH_CrudePrev',
    'SLEEP_CrudePrev',
    'STROKE_CrudePrev',
    'TEETHLOST_CrudePrev'], axis=1)
cond_prevn['lat'] = cond_prevn['Geolocation'].str[1:15]
cond_prevn['lon'] = cond_prevn['Geolocation'].str[-15:-1]
cond_prevn

,StateAbbr,PlaceName,PlaceFIPS,Geo_FIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,...,COREW_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,Geolocation,Year,population_count,lat,lon
450,PA,Pittsburgh,4261000,42003010300,4261000-42003010300,6600.0,12.9,19.0,51.9,11.1,...,30.5,26.5,5.0,17.6,29.8,"(40.43572601140, -79.9851241394)",2019,NaN,40.43572601140,-79.9851241394
451,PA,Pittsburgh,4261000,42003020100,4261000-42003020100,3629.0,6.7,22.1,71.8,9.3,...,40.9,14.4,6.4,24.9,23.4,"(40.44068353060, -80.0000351595)",2019,NaN,40.44068353060,-80.0000351595
452,PA,Pittsburgh,4261000,42003020300,4261000-42003020300,616.0,4.8,19.7,67.6,8.2,...,46.1,12.3,4.9,24.2,23.4,"(40.45425423370, -79.9814998549)",2019,NaN,40.45425423370,-79.9814998549
453,PA,Pittsburgh,4261000,42003030500,4261000-42003030500,2256.0,14.4,48.3,84.3,12.8,...,28.4,26.9,20.5,37.2,42.6,"(40.44273346300, -79.9837719110)",2019,NaN,40.44273346300,-79.9837719110
454,PA,Pittsburgh,4261000,42003040200,4261000-42003040200,2604.0,11.8,23.3,67.1,11.7,...,28.0,22.3,8.0,22.3,29.3,"(40.44011022030, -79.9663476233)",2019,NaN,40.44011022030,-79.9663476233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21696,PA,Pittsburgh,4261000,42003562800,4261000-42003562800,1597.0,13.3,30.2,80.5,10.1,...,26.5,23.6,10.5,34.7,30.4,"(40.4226894882, -80.0713440743)",2016,NaN,"40.4226894882,",-80.0713440743
21700,PA,Pittsburgh,4261000,42003310300,4261000-42003310300,990.0,9.5,30.9,81.3,9.4,...,31.1,19.5,10.1,35.5,28.8,"(40.3879203774, -79.9225535388)",2016,NaN,"40.3879203774,",-79.9225535388
21701,PA,Pittsburgh,4261000,42003050900,4261000-42003050900,1202.0,36.3,47.8,83.4,16.9,...,12.3,37.1,23.7,37.6,51.3,"(40.4510464389, -79.9743932785)",2016,NaN,"40.4510464389,",-79.9743932785
21702,PA,Pittsburgh,4261000,42003070600,4261000-42003070600,2082.0,7.5,19.8,70.8,8.3,...,32.0,12.8,4.7,26.5,20.7,"(40.4532703527, -79.9254775124)",2016,NaN,"40.4532703527,",-79.9254775124


In [59]:
cond_prevn.shape
cond_prevn.isna().sum()

StateAbbr                 0
PlaceName                 0
PlaceFIPS                 0
Geo_FIPS                  0
Place_TractID             0
Population2010          127
ACCESS2_CrudePrev         0
BPHIGH_CrudePrev          0
BPMED_CrudePrev           0
CASTHMA_CrudePrev         0
CHD_CrudePrev             0
CHECKUP_CrudePrev         0
CHOLSCREEN_CrudePrev      0
COPD_CrudePrev            0
COREM_CrudePrev           0
COREW_CrudePrev           8
CSMOKING_CrudePrev        0
DIABETES_CrudePrev        0
HIGHCHOL_CrudePrev        0
OBESITY_CrudePrev         0
Geolocation               0
Year                      0
population_count        381
lat                       0
lon                       0
dtype: int64

In [61]:
cond_prevn[cond_prevn['Population2010'].isna()]

,StateAbbr,PlaceName,PlaceFIPS,Geo_FIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,...,COREW_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,Geolocation,Year,population_count,lat,lon
21552,PA,Pittsburgh,4261000,42003010300,4261000-42003010300,NaN,14.4,17.1,54.3,10.7,...,20.7,23.2,4.4,20.3,29.1,"(40.43572601140, -79.9851241394)",2017,6600.0,40.43572601140,-79.9851241394
21553,PA,Pittsburgh,4261000,42003020100,4261000-42003020100,NaN,9.0,23.2,74.0,9.6,...,26.8,15.5,7.0,29.4,26.5,"(40.44068353060, -80.0000351595)",2017,3629.0,40.44068353060,-80.0000351595
21554,PA,Pittsburgh,4261000,42003020300,4261000-42003020300,NaN,5.3,20.0,70.2,8.0,...,31.4,11.0,5.0,26.9,25.4,"(40.45425423370, -79.9814998549)",2017,616.0,40.45425423370,-79.9814998549
21555,PA,Pittsburgh,4261000,42003030500,4261000-42003030500,NaN,19.0,49.8,85.3,12.9,...,17.2,25.9,21.9,42.6,45.9,"(40.44273346300, -79.9837719110)",2017,2256.0,40.44273346300,-79.9837719110
21556,PA,Pittsburgh,4261000,42003040200,4261000-42003040200,NaN,16.2,23.1,69.2,12.0,...,18.2,21.8,8.0,26.9,30.4,"(40.44011022030, -79.9663476233)",2017,2604.0,40.44011022030,-79.9663476233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21674,PA,Pittsburgh,4261000,42003563200,4261000-42003563200,NaN,13.9,34.9,78.1,10.4,...,21.4,23.3,12.1,37.7,36.9,"(40.45292913280, -79.9979302476)",2017,2439.0,40.45292913280,-79.9979302476
21675,PA,Pittsburgh,4261000,42003980000,4261000-42003980000,NaN,6.4,25.1,76.0,8.0,...,30.0,12.8,6.8,32.3,25.6,"(40.48825403280, -79.8924052803)",2017,540.0,40.48825403280,-79.8924052803
21676,PA,Pittsburgh,4261000,42003980900,4261000-42003980900,NaN,22.6,29.8,62.2,8.5,...,NaN,34.0,8.7,29.6,42.3,"(40.46654148170, -80.0419472659)",2017,1797.0,40.46654148170,-80.0419472659
21677,PA,Pittsburgh,4261000,42003981800,4261000-42003981800,NaN,22.3,49.4,86.0,10.1,...,13.7,20.1,21.2,49.3,34.5,"(40.48038259110, -79.8972205116)",2017,461.0,40.48038259110,-79.8972205116


In [63]:
cond_prevn = cond_prevn.drop(columns = ['Population2010', 'population_count'])
cond_prevn.isna().sum()

StateAbbr               0
PlaceName               0
PlaceFIPS               0
Geo_FIPS                0
Place_TractID           0
ACCESS2_CrudePrev       0
BPHIGH_CrudePrev        0
BPMED_CrudePrev         0
CASTHMA_CrudePrev       0
CHD_CrudePrev           0
CHECKUP_CrudePrev       0
CHOLSCREEN_CrudePrev    0
COPD_CrudePrev          0
COREM_CrudePrev         0
COREW_CrudePrev         8
CSMOKING_CrudePrev      0
DIABETES_CrudePrev      0
HIGHCHOL_CrudePrev      0
OBESITY_CrudePrev       0
Geolocation             0
Year                    0
lat                     0
lon                     0
dtype: int64

### Issue: There are 275 Allegheny Cenus Tracts that are not in CDC prevelance data

Dropping all non City of Pittsburgh CTs would leave us with 125 rows

In [65]:
testMerge = alCoCTs.merge(cond_prevn, how='inner', left_on='geoid', right_on='Geo_FIPS', validate='one_to_many')
testMerge.isna().sum()

_id                     0
fid                     0
state_fp                0
county_fp               0
tract_ce                0
affgeoid                0
geoid                   0
name                    0
lsad                    0
a_land                  0
a_water                 0
shape_length            0
shape_area              0
StateAbbr               0
PlaceName               0
PlaceFIPS               0
Geo_FIPS                0
Place_TractID           0
ACCESS2_CrudePrev       0
BPHIGH_CrudePrev        0
BPMED_CrudePrev         0
CASTHMA_CrudePrev       0
CHD_CrudePrev           0
CHECKUP_CrudePrev       0
CHOLSCREEN_CrudePrev    0
COPD_CrudePrev          0
COREM_CrudePrev         0
COREW_CrudePrev         8
CSMOKING_CrudePrev      0
DIABETES_CrudePrev      0
HIGHCHOL_CrudePrev      0
OBESITY_CrudePrev       0
Geolocation             0
Year                    0
lat                     0
lon                     0
dtype: int64

In [69]:
prevelence_data = cond_prevn
print(prevelence_data.isna().sum())
prevelence_data = prevelence_data.drop(columns=["COREW_CrudePrev"])
print('\nDropping Column with NAs\n')
print(prevelence_data.isna().sum())
print(len(prevelence_data)/4)
prevelence_data

StateAbbr               0
PlaceName               0
PlaceFIPS               0
Geo_FIPS                0
Place_TractID           0
ACCESS2_CrudePrev       0
BPHIGH_CrudePrev        0
BPMED_CrudePrev         0
CASTHMA_CrudePrev       0
CHD_CrudePrev           0
CHECKUP_CrudePrev       0
CHOLSCREEN_CrudePrev    0
COPD_CrudePrev          0
COREM_CrudePrev         0
COREW_CrudePrev         8
CSMOKING_CrudePrev      0
DIABETES_CrudePrev      0
HIGHCHOL_CrudePrev      0
OBESITY_CrudePrev       0
Geolocation             0
Year                    0
lat                     0
lon                     0
dtype: int64

Dropping Column with NAs

StateAbbr               0
PlaceName               0
PlaceFIPS               0
Geo_FIPS                0
Place_TractID           0
ACCESS2_CrudePrev       0
BPHIGH_CrudePrev        0
BPMED_CrudePrev         0
CASTHMA_CrudePrev       0
CHD_CrudePrev           0
CHECKUP_CrudePrev       0
CHOLSCREEN_CrudePrev    0
COPD_CrudePrev          0
COREM_CrudePrev         

,StateAbbr,PlaceName,PlaceFIPS,Geo_FIPS,Place_TractID,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,...,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,Geolocation,Year,lat,lon
450,PA,Pittsburgh,4261000,42003010300,4261000-42003010300,12.9,19.0,51.9,11.1,2.6,...,4.4,38.8,26.5,5.0,17.6,29.8,"(40.43572601140, -79.9851241394)",2019,40.43572601140,-79.9851241394
451,PA,Pittsburgh,4261000,42003020100,4261000-42003020100,6.7,22.1,71.8,9.3,3.9,...,3.8,50.2,14.4,6.4,24.9,23.4,"(40.44068353060, -80.0000351595)",2019,40.44068353060,-80.0000351595
452,PA,Pittsburgh,4261000,42003020300,4261000-42003020300,4.8,19.7,67.6,8.2,2.8,...,2.8,54.3,12.3,4.9,24.2,23.4,"(40.45425423370, -79.9814998549)",2019,40.45425423370,-79.9814998549
453,PA,Pittsburgh,4261000,42003030500,4261000-42003030500,14.4,48.3,84.3,12.8,9.9,...,10.6,32.5,26.9,20.5,37.2,42.6,"(40.44273346300, -79.9837719110)",2019,40.44273346300,-79.9837719110
454,PA,Pittsburgh,4261000,42003040200,4261000-42003040200,11.8,23.3,67.1,11.7,4.0,...,5.5,31.8,22.3,8.0,22.3,29.3,"(40.44011022030, -79.9663476233)",2019,40.44011022030,-79.9663476233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21696,PA,Pittsburgh,4261000,42003562800,4261000-42003562800,13.3,30.2,80.5,10.1,7.6,...,7.7,34.7,23.6,10.5,34.7,30.4,"(40.4226894882, -80.0713440743)",2016,"40.4226894882,",-80.0713440743
21700,PA,Pittsburgh,4261000,42003310300,4261000-42003310300,9.5,30.9,81.3,9.4,7.2,...,6.5,36.8,19.5,10.1,35.5,28.8,"(40.3879203774, -79.9225535388)",2016,"40.3879203774,",-79.9225535388
21701,PA,Pittsburgh,4261000,42003050900,4261000-42003050900,36.3,47.8,83.4,16.9,10.8,...,14.3,19.9,37.1,23.7,37.6,51.3,"(40.4510464389, -79.9743932785)",2016,"40.4510464389,",-79.9743932785
21702,PA,Pittsburgh,4261000,42003070600,4261000-42003070600,7.5,19.8,70.8,8.3,3.4,...,2.9,40.0,12.8,4.7,26.5,20.7,"(40.4532703527, -79.9254775124)",2016,"40.4532703527,",-79.9254775124


In [71]:
testMerge.dropna().shape

(500, 36)

### Utlization Data

Kinda did some sketchy stuff to join this. Pretty much summing treats NAs as 0. Missing 6 Census Tracts for some reason.

In [73]:
ER_use = pd.read_csv("Allegheny_county_ER_use.csv")
ER_use.head()

,_id,Geo_FIPS,StateFIPS,CountyFIPS,TractFIPS,BlockgroupFIPS,TotalPopEst2015_19ACS,YearOfStartDate,UnderAge1PopEst2015_19ACS,Age0to17PopEst2015_19ACS,...,ED_VisitsPer100Age0to17,NumberLowAcuityED_VisitsAge0to17,LowAcuityED_VisitsPer100Age0to17,NumberAsthmaRelatedED_Visits,AsthmaRelatedED_VisitsPer100Age0to17,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,InjuryRelatedEDVisitsPer100Age0to17,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,AcuteRespiratoryTractInfectionRelatedED_VisitsPer100Age0to17
0,1,420031301002,42,3,130100,2,518,2016.0,9.4,207,...,58.9,62.0,30.0,40.0,19.3,14.0,1.0,0.5,18.0,8.7
1,2,420031301002,42,3,130100,2,518,2017.0,9.4,207,...,62.8,80.0,38.6,54.0,26.1,11.0,1.0,0.5,27.0,13.0
2,3,420031301002,42,3,130100,2,518,2018.0,9.4,207,...,66.7,85.0,41.1,59.0,28.5,13.0,2.0,1.0,19.0,9.2
3,4,420031301002,42,3,130100,2,518,2019.0,9.4,207,...,71.5,96.0,46.4,68.0,32.9,7.0,NaN,0.0,27.0,13.0
4,5,420031301003,42,3,130100,3,489,2016.0,0.0,134,...,53.0,39.0,29.1,13.0,9.7,12.0,2.0,1.5,10.0,7.5


In [134]:
ER_use.shape

(4370, 23)

In [136]:
PC_use = pd.read_csv("Allegheny_county_PC_use.csv")
PC_use = PC_use.fillna(0)

In [138]:
PC_use.shape

(4366, 14)

In [140]:
#merging utilization data for Asthma ER and primary care
util_data = PC_use.merge(ER_use, how = 'inner', left_on = ['Geo_FIPS', 'YearOfContactDate'], right_on = ['Geo_FIPS', 'YearOfStartDate'], validate='one_to_one')
#dropping repeat columns
util_data = util_data.drop(["_id_y","StateFIPS_y","CountyFIPS_y","TractFIPS_y","BlockgroupFIPS_y",
                            "TotalPopEst2015_19ACS_y","Age0to17PopEst2015_19ACS_y"],axis="columns")
util_data

,_id_x,Geo_FIPS,StateFIPS_x,CountyFIPS_x,TractFIPS_x,BlockgroupFIPS_x,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,YearOfContactDate,AllPrimaryCarePatientsAge0to17,...,ED_VisitsPer100Age0to17,NumberLowAcuityED_VisitsAge0to17,LowAcuityED_VisitsPer100Age0to17,NumberAsthmaRelatedED_Visits,AsthmaRelatedED_VisitsPer100Age0to17,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,InjuryRelatedEDVisitsPer100Age0to17,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,AcuteRespiratoryTractInfectionRelatedED_VisitsPer100Age0to17
0,1,420031301002,42,3,130100,2,518,207,2016,82,...,58.9,62.0,30.0,40.0,19.3,14.0,1.0,0.5,18.0,8.7
1,2,420031301002,42,3,130100,2,518,207,2017,94,...,62.8,80.0,38.6,54.0,26.1,11.0,1.0,0.5,27.0,13.0
2,3,420031301002,42,3,130100,2,518,207,2018,89,...,66.7,85.0,41.1,59.0,28.5,13.0,2.0,1.0,19.0,9.2
3,4,420031301002,42,3,130100,2,518,207,2019,86,...,71.5,96.0,46.4,68.0,32.9,7.0,NaN,0.0,27.0,13.0
4,5,420031301003,42,3,130100,3,489,134,2016,53,...,53.0,39.0,29.1,13.0,9.7,12.0,2.0,1.5,10.0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4087,4362,420039818001,42,3,981800,1,207,47,2019,4,...,17.0,2.0,4.3,NaN,0.0,NaN,NaN,0.0,NaN,0.0
4088,4363,420039822001,42,3,982200,1,4618,80,2016,16,...,12.5,2.0,2.5,NaN,0.0,1.0,NaN,0.0,NaN,0.0
4089,4364,420039822001,42,3,982200,1,4618,80,2017,18,...,12.5,2.0,2.5,1.0,1.3,1.0,NaN,0.0,3.0,3.8
4090,4365,420039822001,42,3,982200,1,4618,80,2018,19,...,11.3,2.0,2.5,NaN,0.0,2.0,NaN,0.0,3.0,3.8


In [142]:
#Joining rows based on CTs
#Dropping Per100 columns
util_data = drop_columns_with_keyword(util_data,'Per100')
util_data['Geo_FIPS'] = util_data['Geo_FIPS'].apply(str)
util_data['Geo_FIPS'] = util_data['Geo_FIPS'].str[:-1]
util_data = util_data.drop(columns=['_id_x', 'StateFIPS_x', 'CountyFIPS_x', 'TractFIPS_x', 'BlockgroupFIPS_x'])
util_data = util_data.groupby(['Geo_FIPS', 'YearOfContactDate'], as_index=False).sum()
util_data.isna().sum()

Geo_FIPS                                                        0
YearOfContactDate                                               0
TotalPopEst2015_19ACS_x                                         0
Age0to17PopEst2015_19ACS_x                                      0
AllPrimaryCarePatientsAge0to17                                  0
WellChildVisitsInPastYearAge0to17                               0
AsthmaDiagnosisAge0to17                                         0
YearOfStartDate                                                 0
UnderAge1PopEst2015_19ACS                                       0
Unique0to17WithED_Visit                                         0
NumberED_VisitsAge0to17                                         0
NumberLowAcuityED_VisitsAge0to17                                0
NumberAsthmaRelatedED_Visits                                    0
NumberED_VisitsByChildrenUnder1YearOld                          0
NumberOfInjuryRelatedVisits                                     0
NumberAcut

In [144]:
len(util_data[util_data['YearOfContactDate'] == 2019])

366

### Race Data

In [147]:
#loading racial data from 2010 census
Race_data = pd.read_csv("2010_census_race_P9-Data.csv", header = 1)
#removing extra details on multi-racial individuals
Race_data = Race_data.iloc[:,:13]
Race_data['Type'] = Race_data['Geographic Area Name'].apply(lambda x : ' '.join(x.split(' ')[:2]))
Race_data = Race_data[Race_data['Type'] == 'Census Tract']
Race_data

,Geography,Geographic Area Name,Total,Total!!Hispanic or Latino,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!Population of one race,Total!!Not Hispanic or Latino!!Population of one race!!White alone,Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone,Total!!Not Hispanic or Latino!!Population of one race!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Population of one race!!Asian alone,Total!!Not Hispanic or Latino!!Population of one race!!Native Hawaiian and Other Pacific Islander alone,Total!!Not Hispanic or Latino!!Population of one race!!Some Other Race alone,Total!!Not Hispanic or Latino!!Two or More Races,Type
1,1400000US42003010300,"Census Tract 103, Allegheny County, Pennsylvania",6600,161,6439,6362,4260,2004,12,72,2,12,77,Census Tract
2,1400000US42003020100,"Census Tract 201, Allegheny County, Pennsylvania",3629,119,3510,3422,2642,573,8,171,1,27,88,Census Tract
3,1400000US42003020300,"Census Tract 203, Allegheny County, Pennsylvania",616,10,606,595,459,96,1,39,0,0,11,Census Tract
4,1400000US42003030500,"Census Tract 305, Allegheny County, Pennsylvania",2256,32,2224,2179,298,1846,7,24,0,4,45,Census Tract
5,1400000US42003040200,"Census Tract 402, Allegheny County, Pennsylvania",2604,61,2543,2505,1331,997,7,169,0,1,38,Census Tract
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,1400000US42003981000,"Census Tract 9810, Allegheny County, Pennsylvania",4,0,4,4,4,0,0,0,0,0,0,Census Tract
399,1400000US42003981100,"Census Tract 9811, Allegheny County, Pennsylvania",7,0,7,7,6,1,0,0,0,0,0,Census Tract
400,1400000US42003981200,"Census Tract 9812, Allegheny County, Pennsylvania",0,0,0,0,0,0,0,0,0,0,0,Census Tract
401,1400000US42003981800,"Census Tract 9818, Allegheny County, Pennsylvania",461,14,447,436,257,174,3,2,0,0,11,Census Tract


In [149]:
Race_data.shape

(402, 14)

### House Hold Income Data

In [152]:
#loading 5 year HH income data adj to 2016 inflation levels
HHincome_data = pd.read_csv("2016_ad_HHincome.S2503-Data.csv", na_values = ['-'])
#removed data on Housing cost as % of hh income
#HHincome_data = HHincome_data.iloc[:,:-133]
HHincome_data['Type'] = HHincome_data['Geographic Area Name'].apply(lambda x : ' '.join(x.split(' ')[:2]))
HHincome_data = HHincome_data[HHincome_data['Type'] == 'Census Tract']
HHincome_data

,Geography,Geographic Area Name,Occupied housing units!!Estimate!!Occupied housing units,Occupied housing units!!Margin of Error!!Occupied housing units,Owner-occupied housing units!!Estimate!!Occupied housing units,Owner-occupied housing units!!Margin of Error!!Occupied housing units,Renter-occupied housing units!!Estimate!!Occupied housing units,Renter-occupied housing units!!Margin of Error!!Occupied housing units,"Occupied housing units!!Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000","Occupied housing units!!Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000",...,Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Zero or negative income,Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Zero or negative income,Renter-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Zero or negative income,Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!No cash rent,Occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!No cash rent,Owner-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!No cash rent,Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!No cash rent,Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!No cash rent,Renter-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!No cash rent,Type
1,1400000US42003010300,"Census Tract 103, Allegheny County, Pennsylvania",281,38,73,31,208,44,4.6,5.1,...,26,2.9,4.3,0.0,7.7,(X),(X),0.0,10.2,Census Tract
2,1400000US42003020100,"Census Tract 201, Allegheny County, Pennsylvania",2042,160,408,116,1634,154,4.8,2.5,...,5.4,2.6,2.1,0.0,1.1,(X),(X),0.0,1.4,Census Tract
3,1400000US42003020300,"Census Tract 203, Allegheny County, Pennsylvania",553,48,86,37,467,57,2.7,3,...,22.7,3.2,3.5,0.0,4,(X),(X),0.0,4.7,Census Tract
4,1400000US42003030500,"Census Tract 305, Allegheny County, Pennsylvania",1117,96,211,51,906,116,8.3,6.1,...,10.1,3.6,4.4,0.0,2,(X),(X),0.0,2.4,Census Tract
5,1400000US42003040200,"Census Tract 402, Allegheny County, Pennsylvania",611,61,182,51,429,69,18.7,8.6,...,11.6,16.6,10,1.5,1.6,(X),(X),2.1,2.2,Census Tract
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,1400000US42003981000,"Census Tract 9810, Allegheny County, Pennsylvania",0,10,0,10,0,10,NaN,**,...,**,NaN,**,NaN,**,(X),(X),NaN,**,Census Tract
399,1400000US42003981100,"Census Tract 9811, Allegheny County, Pennsylvania",5,7,0,10,5,7,0.0,100,...,**,0.0,100,0.0,100,(X),(X),0.0,100,Census Tract
400,1400000US42003981200,"Census Tract 9812, Allegheny County, Pennsylvania",0,10,0,10,0,10,NaN,**,...,**,NaN,**,NaN,**,(X),(X),NaN,**,Census Tract
401,1400000US42003981800,"Census Tract 9818, Allegheny County, Pennsylvania",17,10,5,7,12,11,0.0,59.8,...,100,0.0,71.2,0.0,59.8,(X),(X),0.0,71.2,Census Tract


In [155]:
HHincome_data = drop_columns_with_keyword(HHincome_data,'Margin of Error')
HHincome_data = drop_columns_with_keyword(HHincome_data,'Owner-occupied')
HHincome_data = drop_columns_with_keyword(HHincome_data,'Renter-occupied')
HHincome_data = HHincome_data.iloc[:,:15]
colsDict = {}
for col in HHincome_data.columns[3:]:
    colsDict[col] = '!!'.join(col.split('!!')[2:4])
HHincome_data = HHincome_data.rename(columns=colsDict)
HHincome_data

,Geography,Geographic Area Name,Occupied housing units!!Estimate!!Occupied housing units,"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$5,000 to $9,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$10,000 to $14,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$15,000 to $19,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $24,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$25,000 to $34,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$35,000 to $49,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$50,000 to $74,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$75,000 to $99,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$100,000 to $149,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)
1,1400000US42003010300,"Census Tract 103, Allegheny County, Pennsylvania",281,4.6,15.3,2.8,21.4,8.5,5.0,14.2,19.6,1.8,4.6,2.1,22292.0
2,1400000US42003020100,"Census Tract 201, Allegheny County, Pennsylvania",2042,4.8,7.3,4.2,4.7,4.6,9.2,5.4,12.3,8.2,13.3,26.1,72642.0
3,1400000US42003020300,"Census Tract 203, Allegheny County, Pennsylvania",553,2.7,0.0,2.5,3.6,2.7,4.0,6.5,16.8,14.6,17.7,28.8,93125.0
4,1400000US42003030500,"Census Tract 305, Allegheny County, Pennsylvania",1117,8.3,15.0,15.0,17.2,7.5,9.3,7.0,11.7,3.8,3.2,1.8,18730.0
5,1400000US42003040200,"Census Tract 402, Allegheny County, Pennsylvania",611,18.7,22.3,6.2,9.2,10.6,5.4,16.9,4.7,3.3,1.3,1.5,16250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,1400000US42003981000,"Census Tract 9810, Allegheny County, Pennsylvania",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,1400000US42003981100,"Census Tract 9811, Allegheny County, Pennsylvania",5,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,NaN
400,1400000US42003981200,"Census Tract 9812, Allegheny County, Pennsylvania",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,1400000US42003981800,"Census Tract 9818, Allegheny County, Pennsylvania",17,0.0,23.5,0.0,52.9,23.5,0.0,0.0,0.0,0.0,0.0,0.0,18750.0


In [157]:
#Merging census datasets
census_data = Race_data.merge(HHincome_data, how = 'left', left_on = 'Geography', right_on = 'Geography')
#census_data = census_data.drop(index=0)
census_data['Geo_FIPS'] = census_data['Geography'].str[-11:]
#col = census_data.pop('Geo_FIPS')
#census_data.insert(0, 'Geo_FIPS', col)
census_data

,Geography,Geographic Area Name_x,Total,Total!!Hispanic or Latino,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!Population of one race,Total!!Not Hispanic or Latino!!Population of one race!!White alone,Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone,Total!!Not Hispanic or Latino!!Population of one race!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Population of one race!!Asian alone,...,"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$15,000 to $19,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $24,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$25,000 to $34,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$35,000 to $49,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$50,000 to $74,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$75,000 to $99,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$100,000 to $149,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars),Geo_FIPS
0,1400000US42003010300,"Census Tract 103, Allegheny County, Pennsylvania",6600,161,6439,6362,4260,2004,12,72,...,21.4,8.5,5.0,14.2,19.6,1.8,4.6,2.1,22292.0,42003010300
1,1400000US42003020100,"Census Tract 201, Allegheny County, Pennsylvania",3629,119,3510,3422,2642,573,8,171,...,4.7,4.6,9.2,5.4,12.3,8.2,13.3,26.1,72642.0,42003020100
2,1400000US42003020300,"Census Tract 203, Allegheny County, Pennsylvania",616,10,606,595,459,96,1,39,...,3.6,2.7,4.0,6.5,16.8,14.6,17.7,28.8,93125.0,42003020300
3,1400000US42003030500,"Census Tract 305, Allegheny County, Pennsylvania",2256,32,2224,2179,298,1846,7,24,...,17.2,7.5,9.3,7.0,11.7,3.8,3.2,1.8,18730.0,42003030500
4,1400000US42003040200,"Census Tract 402, Allegheny County, Pennsylvania",2604,61,2543,2505,1331,997,7,169,...,9.2,10.6,5.4,16.9,4.7,3.3,1.3,1.5,16250.0,42003040200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,1400000US42003981000,"Census Tract 9810, Allegheny County, Pennsylvania",4,0,4,4,4,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42003981000
398,1400000US42003981100,"Census Tract 9811, Allegheny County, Pennsylvania",7,0,7,7,6,1,0,0,...,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,NaN,42003981100
399,1400000US42003981200,"Census Tract 9812, Allegheny County, Pennsylvania",0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42003981200
400,1400000US42003981800,"Census Tract 9818, Allegheny County, Pennsylvania",461,14,447,436,257,174,3,2,...,52.9,23.5,0.0,0.0,0.0,0.0,0.0,0.0,18750.0,42003981800


In [159]:
census_data.isna().sum()

Geography                                                                                                          0
Geographic Area Name_x                                                                                             0
Total                                                                                                              0
Total!!Hispanic or Latino                                                                                          0
Total!!Not Hispanic or Latino                                                                                      0
Total!!Not Hispanic or Latino!!Population of one race                                                              0
Total!!Not Hispanic or Latino!!Population of one race!!White alone                                                 0
Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone                             0
Total!!Not Hispanic or Latino!!Population of one race!!American 

In [191]:
#Loading Emissions data
emissions_data = pd.read_csv("Emissions_Data_w_Census_Tracts_Pivot.csv")
# Retaining data from 2016-2019 to align with utilization data
#emissions_data = emissions_data[(emissions_data["year"] <= 2019) & (emissions_data["year"] >= 2016)]

emissions_data = alCoCTs[['geoid', 'tract_ce']].merge(emissions_data, how = 'inner', left_on='tract_ce', right_on='TractCE', validate='one_to_many')
emissions_data=emissions_data.fillna(0)

In [193]:
alCoCTs['geoid'].unique

<bound method Series.unique of 0      42003050900
1      42003070300
2      42003120700
3      42003140400
4      42003180700
          ...     
397    42003141000
398    42003202300
399    42003320600
400    42003408001
401    42003414101
Name: geoid, Length: 402, dtype: int64>

## Final Merges

In [206]:
#Merging health care utlization data with census data on race and household income
utilxcensus_data = util_data.merge(census_data, how='inner', left_on='Geo_FIPS', right_on='Geo_FIPS')
utilxcensus_data['Geo_FIPS']=utilxcensus_data['Geo_FIPS'].apply(int)
utilxcensus_data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,...,"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$10,000 to $14,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$15,000 to $19,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $24,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$25,000 to $34,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$35,000 to $49,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$50,000 to $74,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$75,000 to $99,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$100,000 to $149,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)
0,42003100500,2016,1845,275,156,114,36.0,4032.0,18.6,120.0,...,3.6,8.3,6.5,10.7,17.1,17.3,12.6,8.0,6.0,46308.0
1,42003100500,2017,1845,275,158,122,38.0,4034.0,18.6,121.0,...,3.6,8.3,6.5,10.7,17.1,17.3,12.6,8.0,6.0,46308.0
2,42003100500,2018,1845,275,176,142,36.0,4036.0,18.6,115.0,...,3.6,8.3,6.5,10.7,17.1,17.3,12.6,8.0,6.0,46308.0
3,42003100500,2019,1845,275,166,140,34.0,4038.0,18.6,117.0,...,3.6,8.3,6.5,10.7,17.1,17.3,12.6,8.0,6.0,46308.0
4,42003101100,2016,2573,380,102,80,20.0,6048.0,16.6,82.0,...,6.7,5.8,4.6,18.6,14.5,22.4,12.3,5.8,1.1,37500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,42003981800,2019,207,47,4,3,2.0,2019.0,0.0,4.0,...,0.0,52.9,23.5,0.0,0.0,0.0,0.0,0.0,0.0,18750.0
1461,42003982200,2016,4618,80,16,13,1.0,2016.0,0.0,6.0,...,18.2,18.2,13.6,0.0,0.0,0.0,0.0,0.0,18.2,16250.0
1462,42003982200,2017,4618,80,18,16,1.0,2017.0,0.0,10.0,...,18.2,18.2,13.6,0.0,0.0,0.0,0.0,0.0,18.2,16250.0
1463,42003982200,2018,4618,80,19,14,1.0,2018.0,0.0,7.0,...,18.2,18.2,13.6,0.0,0.0,0.0,0.0,0.0,18.2,16250.0


508

In [218]:
#merged census and utilization data with health condition prevalence data
utilxcensus_data['YearOfContactDate'] = utilxcensus_data['YearOfContactDate'].astype(int)
prevelence_data['Year']= prevelence_data['Year'].astype(int)
semi_data = utilxcensus_data.merge(prevelence_data, how='left', left_on=['Geo_FIPS', 'YearOfContactDate'], right_on=['Geo_FIPS','Year'])
#semi_data['TRACTCE'] = semi_data['TractFIPS_x']
semi_data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,...,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,Geolocation,Year,lat,lon
0,42003100500,2016,1845,275,156,114,36.0,4032.0,18.6,120.0,...,8.4,28.3,22.8,17.4,37.1,39.7,"(40.4771301309, -79.9369806524)",2016.0,"40.4771301309,",-79.9369806524
1,42003100500,2017,1845,275,158,122,38.0,4034.0,18.6,121.0,...,7.3,28.3,18.5,15.9,39.6,39.3,"(40.47713013090, -79.9369806524)",2017.0,40.47713013090,-79.9369806524
2,42003100500,2018,1845,275,176,142,36.0,4036.0,18.6,115.0,...,7.3,42.4,19.1,14.9,39.6,36.5,POINT (-79.9369806524 40.4771301309),2018.0,OINT (-79.9369,40.4771301309
3,42003100500,2019,1845,275,166,140,34.0,4038.0,18.6,117.0,...,6.9,42.4,18.8,14.3,34.7,35.2,"(40.47713013090, -79.9369806524)",2019.0,40.47713013090,-79.9369806524
4,42003101100,2016,2573,380,102,80,20.0,6048.0,16.6,82.0,...,6.7,34.0,24.6,9.2,31.3,32.3,"(40.4838814154, -79.9497580289)",2016.0,"40.4838814154,",-79.9497580289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,42003981800,2019,207,47,4,3,2.0,2019.0,0.0,4.0,...,10.6,34.8,16.3,17.4,39.9,29.0,"(40.48038259110, -79.8972205116)",2019.0,40.48038259110,-79.8972205116
1461,42003982200,2016,4618,80,16,13,1.0,2016.0,0.0,6.0,...,4.4,25.4,30.7,1.7,9.7,24.5,"(40.4444878719, -79.9547651976)",2016.0,"40.4444878719,",-79.9547651976
1462,42003982200,2017,4618,80,18,16,1.0,2017.0,0.0,10.0,...,4.2,25.4,24.3,1.4,11.1,24.3,"(40.44448787190, -79.9547651976)",2017.0,40.44448787190,-79.9547651976
1463,42003982200,2018,4618,80,19,14,1.0,2018.0,0.0,7.0,...,4.8,32.6,26.4,1.9,11.1,25.5,POINT (-79.9547651976 40.4444878719),2018.0,OINT (-79.9547,40.4444878719


In [220]:
#joining emissions dataset
data = semi_data.merge(emissions_data, how='left', left_on=['Geo_FIPS', 'YearOfContactDate'], right_on=['geoid','Year'])
#dropping duplicate columns
#data = data.drop(['TARGET_FID','STATEFP','COUNTYFP','TRACTCE','AFFGEOID','GEOID','NAME'],axis=1)
data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,...,Styrene,Sulfur Oxides,Sulfuric Acid,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Volatile Organic Compounds,Xylenes (Isomers And Mixture),Grand Total
0,42003100500,2016,1845,275,156,114,36.0,4032.0,18.6,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42003100500,2017,1845,275,158,122,38.0,4034.0,18.6,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42003100500,2018,1845,275,176,142,36.0,4036.0,18.6,115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42003100500,2019,1845,275,166,140,34.0,4038.0,18.6,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42003101100,2016,2573,380,102,80,20.0,6048.0,16.6,82.0,...,0.0,0.00,0.0,0.0,0.14,0.0,0.0,23.50,0.07,24.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,42003981800,2019,207,47,4,3,2.0,2019.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,42003982200,2016,4618,80,16,13,1.0,2016.0,0.0,6.0,...,0.0,0.31,0.0,0.0,0.00,0.0,0.0,2.85,0.00,62280.4204
1462,42003982200,2017,4618,80,18,16,1.0,2017.0,0.0,10.0,...,0.0,0.31,0.0,0.0,0.00,0.0,0.0,2.87,0.00,62926.7404
1463,42003982200,2018,4618,80,19,14,1.0,2018.0,0.0,7.0,...,0.0,0.36,0.0,0.0,0.00,0.0,0.0,3.31,0.00,72196.5405


In [224]:
extraCols = ['Geography', 
        'Geographic Area Name_x',
        'Type',
        'Geographic Area Name_y',
        'StateAbbr',
        'PlaceName',
        'PlaceFIPS',
        #'Place_TractID',
        'Geolocation',
        'lat',
        'lon',
        'geoid',
        'tract_ce',
        #'Year',
        'TractCE']
data = data.drop(columns = extraCols)
data

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,...,Styrene,Sulfur Oxides,Sulfuric Acid,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Volatile Organic Compounds,Xylenes (Isomers And Mixture),Grand Total
0,42003100500,2016,1845,275,156,114,36.0,4032.0,18.6,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42003100500,2017,1845,275,158,122,38.0,4034.0,18.6,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42003100500,2018,1845,275,176,142,36.0,4036.0,18.6,115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42003100500,2019,1845,275,166,140,34.0,4038.0,18.6,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42003101100,2016,2573,380,102,80,20.0,6048.0,16.6,82.0,...,0.0,0.00,0.0,0.0,0.14,0.0,0.0,23.50,0.07,24.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,42003981800,2019,207,47,4,3,2.0,2019.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,42003982200,2016,4618,80,16,13,1.0,2016.0,0.0,6.0,...,0.0,0.31,0.0,0.0,0.00,0.0,0.0,2.85,0.00,62280.4204
1462,42003982200,2017,4618,80,18,16,1.0,2017.0,0.0,10.0,...,0.0,0.31,0.0,0.0,0.00,0.0,0.0,2.87,0.00,62926.7404
1463,42003982200,2018,4618,80,19,14,1.0,2018.0,0.0,7.0,...,0.0,0.36,0.0,0.0,0.00,0.0,0.0,3.31,0.00,72196.5405


In [477]:
pd.set_option('display.max_rows', 125)
data.isna().sum()

Geo_FIPS                                                                                                            0
TotalPopEst2015_19ACS_x                                                                                             6
Age0to17PopEst2015_19ACS_x                                                                                          6
AllPrimaryCarePatientsAge0to17                                                                                      6
WellChildVisitsInPastYearAge0to17                                                                                   6
AsthmaDiagnosisAge0to17                                                                                             6
YearOfStartDate                                                                                                     6
UnderAge1PopEst2015_19ACS                                                                                           6
Unique0to17WithED_Visit                                 

In [479]:
pd.set_option('display.max_columns', 125)
data[data['TotalPopEst2015_19ACS_x'].isna()]

,Geo_FIPS,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,NumberED_VisitsAge0to17,NumberLowAcuityED_VisitsAge0to17,NumberAsthmaRelatedED_Visits,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,Total,Total!!Hispanic or Latino,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!Population of one race,Total!!Not Hispanic or Latino!!Population of one race!!White alone,Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone,Total!!Not Hispanic or Latino!!Population of one race!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Population of one race!!Asian alone,Total!!Not Hispanic or Latino!!Population of one race!!Native Hawaiian and Other Pacific Islander alone,Total!!Not Hispanic or Latino!!Population of one race!!Some Other Race alone,Total!!Not Hispanic or Latino!!Two or More Races,Occupied housing units!!Estimate!!Occupied housing units,"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$5,000 to $9,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$10,000 to $14,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$15,000 to $19,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $24,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$25,000 to $34,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$35,000 to $49,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$50,000 to $74,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$75,000 to $99,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$100,000 to $149,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars),Place_TractID,Population2010,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,"1,1,2-Trichloroethane","1,1,2,2-Tetrachloroethane","1,2-Ethanediol (Ethylene Glycol)","1,3-Butadiene","1,4-Dichlorobenzene","1,4-Dioxane (1,4-Diethyleneoxide)","2,2,4-Trimethylpentane",Acetaldehyde,Acrolein,Acrylic Acid,Acrylonitrile,Antimony,Arsenic,Benzene,Benzyl Chloride,Cadmium,Carbon Disulfide,Carbon Tetrachloride,Chlorine,Chlorobenzene,Chloroethene (vinyl chloride),Chloroform,Chromium,Cobalt,Cresols/Cresylic Acid (Isomers And Mixture),Cumene,Cyanides,Dibutylphthalate,Diethanolamine,"Epichlorohydrin (1-Chloro-2,3-Epoxypropane)",Ethyl Chloride (Chloroethane),Ethylbenzene,"Ethylene Dichloride (1,2-Dichloroethane)",Ethylene Oxide,"Ethylidene Dichloride (1,1-Dichloroethane)",Formaldehyde,Glycol Ethers,Hexane,Hexavalent Chromium,Hydrochloric Acid,Hydrogen Fluoride (Hydrofluoric Acid),Isophorone,Lead,m-Xylene,Maleic Anhydride,Manganese,Mercury,Methanol,Methyl Chloride (Chloromethane),"Methyl Chloroform (1,1,1-Trichloroethane)",Methyl Isobutyl Ketone (4-Methyl-2-Pentanone),Methyl Methacrylate,Methylene Chloride (Dichloromethane),Naphthalene,Nickel,o-Xylene,Phenol,Phosphorus,Phthalic Anhydride,Polycyclic Organic Matter,Propionaldehyde,Propylene Dichloride,Propylene Oxide,Selenium,Styrene,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Xylenes (Isomers And Mixture),Grand Total
388,42003980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [228]:
cityData = data[data['Place_TractID'].isna()==False]
cityData

,Geo_FIPS,YearOfContactDate,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,...,Styrene,Sulfur Oxides,Sulfuric Acid,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Volatile Organic Compounds,Xylenes (Isomers And Mixture),Grand Total
0,42003100500,2016,1845,275,156,114,36.0,4032.0,18.6,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42003100500,2017,1845,275,158,122,38.0,4034.0,18.6,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42003100500,2018,1845,275,176,142,36.0,4036.0,18.6,115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42003100500,2019,1845,275,166,140,34.0,4038.0,18.6,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42003101100,2016,2573,380,102,80,20.0,6048.0,16.6,82.0,...,0.0,0.00,0.0,0.0,0.14,0.0,0.0,23.50,0.07,24.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,42003981800,2019,207,47,4,3,2.0,2019.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,42003982200,2016,4618,80,16,13,1.0,2016.0,0.0,6.0,...,0.0,0.31,0.0,0.0,0.00,0.0,0.0,2.85,0.00,62280.4204
1462,42003982200,2017,4618,80,18,16,1.0,2017.0,0.0,10.0,...,0.0,0.31,0.0,0.0,0.00,0.0,0.0,2.87,0.00,62926.7404
1463,42003982200,2018,4618,80,19,14,1.0,2018.0,0.0,7.0,...,0.0,0.36,0.0,0.0,0.00,0.0,0.0,3.31,0.00,72196.5405


In [483]:
cityData.isna().sum()

Geo_FIPS                                                                                                          0
TotalPopEst2015_19ACS_x                                                                                           2
Age0to17PopEst2015_19ACS_x                                                                                        2
AllPrimaryCarePatientsAge0to17                                                                                    2
WellChildVisitsInPastYearAge0to17                                                                                 2
AsthmaDiagnosisAge0to17                                                                                           2
YearOfStartDate                                                                                                   2
UnderAge1PopEst2015_19ACS                                                                                         2
Unique0to17WithED_Visit                                                 

In [485]:
cityData[cityData['TotalPopEst2015_19ACS_x'].isna() == True]

,Geo_FIPS,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,NumberED_VisitsAge0to17,NumberLowAcuityED_VisitsAge0to17,NumberAsthmaRelatedED_Visits,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,Total,Total!!Hispanic or Latino,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!Population of one race,Total!!Not Hispanic or Latino!!Population of one race!!White alone,Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone,Total!!Not Hispanic or Latino!!Population of one race!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Population of one race!!Asian alone,Total!!Not Hispanic or Latino!!Population of one race!!Native Hawaiian and Other Pacific Islander alone,Total!!Not Hispanic or Latino!!Population of one race!!Some Other Race alone,Total!!Not Hispanic or Latino!!Two or More Races,Occupied housing units!!Estimate!!Occupied housing units,"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$5,000 to $9,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$10,000 to $14,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$15,000 to $19,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $24,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$25,000 to $34,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$35,000 to $49,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$50,000 to $74,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$75,000 to $99,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$100,000 to $149,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars),Place_TractID,Population2010,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,"1,1,2-Trichloroethane","1,1,2,2-Tetrachloroethane","1,2-Ethanediol (Ethylene Glycol)","1,3-Butadiene","1,4-Dichlorobenzene","1,4-Dioxane (1,4-Diethyleneoxide)","2,2,4-Trimethylpentane",Acetaldehyde,Acrolein,Acrylic Acid,Acrylonitrile,Antimony,Arsenic,Benzene,Benzyl Chloride,Cadmium,Carbon Disulfide,Carbon Tetrachloride,Chlorine,Chlorobenzene,Chloroethene (vinyl chloride),Chloroform,Chromium,Cobalt,Cresols/Cresylic Acid (Isomers And Mixture),Cumene,Cyanides,Dibutylphthalate,Diethanolamine,"Epichlorohydrin (1-Chloro-2,3-Epoxypropane)",Ethyl Chloride (Chloroethane),Ethylbenzene,"Ethylene Dichloride (1,2-Dichloroethane)",Ethylene Oxide,"Ethylidene Dichloride (1,1-Dichloroethane)",Formaldehyde,Glycol Ethers,Hexane,Hexavalent Chromium,Hydrochloric Acid,Hydrogen Fluoride (Hydrofluoric Acid),Isophorone,Lead,m-Xylene,Maleic Anhydride,Manganese,Mercury,Methanol,Methyl Chloride (Chloromethane),"Methyl Chloroform (1,1,1-Trichloroethane)",Methyl Isobutyl Ketone (4-Methyl-2-Pentanone),Methyl Methacrylate,Methylene Chloride (Dichloromethane),Naphthalene,Nickel,o-Xylene,Phenol,Phosphorus,Phthalic Anhydride,Polycyclic Organic Matter,Propionaldehyde,Propylene Dichloride,Propylene Oxide,Selenium,Styrene,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Xylenes (Isomers And Mixture),Grand Total
388,42003980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [495]:
data['Geo_FIPS'] = data['Geo_FIPS'].astype(str)
data['TRACT'] = data['Geo_FIPS'].apply(lambda x:int(x[5:]))
data

,Geo_FIPS,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,AsthmaDiagnosisAge0to17,YearOfStartDate,UnderAge1PopEst2015_19ACS,Unique0to17WithED_Visit,NumberED_VisitsAge0to17,NumberLowAcuityED_VisitsAge0to17,NumberAsthmaRelatedED_Visits,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,Total,Total!!Hispanic or Latino,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!Population of one race,Total!!Not Hispanic or Latino!!Population of one race!!White alone,Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone,Total!!Not Hispanic or Latino!!Population of one race!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Population of one race!!Asian alone,Total!!Not Hispanic or Latino!!Population of one race!!Native Hawaiian and Other Pacific Islander alone,Total!!Not Hispanic or Latino!!Population of one race!!Some Other Race alone,Total!!Not Hispanic or Latino!!Two or More Races,Occupied housing units!!Estimate!!Occupied housing units,"HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$5,000 to $9,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$10,000 to $14,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$15,000 to $19,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $24,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$25,000 to $34,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$35,000 to $49,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$50,000 to $74,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$75,000 to $99,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$100,000 to $149,999","HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars),Place_TractID,Population2010,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,"1,1,2-Trichloroethane","1,1,2,2-Tetrachloroethane","1,2-Ethanediol (Ethylene Glycol)","1,3-Butadiene","1,4-Dichlorobenzene","1,4-Dioxane (1,4-Diethyleneoxide)","2,2,4-Trimethylpentane",Acetaldehyde,...,Acrylonitrile,Antimony,Arsenic,Benzene,Benzyl Chloride,Cadmium,Carbon Disulfide,Carbon Tetrachloride,Chlorine,Chlorobenzene,Chloroethene (vinyl chloride),Chloroform,Chromium,Cobalt,Cresols/Cresylic Acid (Isomers And Mixture),Cumene,Cyanides,Dibutylphthalate,Diethanolamine,"Epichlorohydrin (1-Chloro-2,3-Epoxypropane)",Ethyl Chloride (Chloroethane),Ethylbenzene,"Ethylene Dichloride (1,2-Dichloroethane)",Ethylene Oxide,"Ethylidene Dichloride (1,1-Dichloroethane)",Formaldehyde,Glycol Ethers,Hexane,Hexavalent Chromium,Hydrochloric Acid,Hydrogen Fluoride (Hydrofluoric Acid),Isophorone,Lead,m-Xylene,Maleic Anhydride,Manganese,Mercury,Methanol,Methyl Chloride (Chloromethane),"Methyl Chloroform (1,1,1-Trichloroethane)",Methyl Isobutyl Ketone (4-Methyl-2-Pentanone),Methyl Methacrylate,Methylene Chloride (Dichloromethane),Naphthalene,Nickel,o-Xylene,Phenol,Phosphorus,Phthalic Anhydride,Polycyclic Organic Matter,Propionaldehyde,Propylene Dichloride,Propylene Oxide,Selenium,Styrene,Tetrachloroethylene (Perchloroethylene),Toluene,Trichloroethylene,"Vinylidene Chloride (1,1-Dichloroethylene)",Xylenes (Isomers And Mixture),Grand Total,TRACT
0,42003010300,4147.0,155.0,50.0,33.0,9.0,0.0,0.0,0.0,0.0,0.0,0.